In [ ]:
!pip install riotwatcher

In [2]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd

In [58]:
API_KEY = "RGAPI-c7cf2424-731f-4a9a-8f1f-f6985e62dac3"
lol_watcher = LolWatcher(API_KEY)


TIERS = ["IRON", "SILVER", "GOLD", "PLATINUM", "DIAMOND"]
RANKS = ["I", "II", "III", "IV"]
REGIONS = ["eun1"]
PAGES = [1]

In [59]:
player = lol_watcher.summoner.by_name('eun1', 'taehyungbottoms')

In [60]:
player

{'id': 'UyY4PU4F8nlC6um4bcKbcIjNJccLlbV7MH5DWI1GHW5ybzw',
 'accountId': '-Klgb4_SGuh085cjal9Ah_auxVKRig857l_U0H7hdReKiti4Q32UdjBE',
 'puuid': 'B4hSeGmRWzewpWhcz9cFdWYFsS21kIb0ZLB30JXAb2qPAu7aPHVy97WUrMDcSzvhTT2XrOgfxckN5g',
 'name': 'taehyungbottoms',
 'profileIconId': 5750,
 'revisionDate': 1695834086000,
 'summonerLevel': 95}

In [61]:
ranked_stats = lol_watcher.league.by_summoner('eun1', player['id'])

In [62]:
ranked_stats

[]

In [68]:
columns = [
    "name",
    "summonerLevel",
    "tierRank",
    "wins",
    "losses",
    "veteran",
    "inactive",
    "freshBlood",
    "hotStreak"
]

In [75]:
data = {}
# player = lol_watcher.summoner.by_name('eun1', 'spikal')
data['name'] = player['name']
data['summonerLevel'] = player['summonerLevel']
if len(ranked_stats) > 0:
  for i in range(0,len(ranked_stats)):
    if 'RANKED_SOLO_5x5' in ranked_stats[i]['queueType']:
      if 'tier' in ranked_stats[i] and 'rank' in ranked_stats[i]:
        t = ranked_stats[i]['tier'] + ' ' + ranked_stats[i]['rank']
        data['tierRank'] = t
      else:
        data['tierRank'] = None

      if 'wins' in ranked_stats[i]:
        t = ranked_stats[i]['wins']
        data['wins'] = t
      else:
        data['wins'] = None

      if 'losses' in ranked_stats[i]:
        t = ranked_stats[i]['losses']
        data['losses'] = t
      else:
        data['losses'] = None

      if 'veteran' in ranked_stats[i]:
        t = ranked_stats[i]['veteran']
        data['veteran'] = t
      else:
        data['veteran'] = None

      if 'inactive' in ranked_stats[i]:
        t = ranked_stats[i]['inactive']
        data['inactive'] = t
      else:
        data['inactive'] = None

      if 'freshBlood' in ranked_stats[i]:
        t = ranked_stats[i]['freshBlood']
        data['freshBlood'] = t
      else:
        data['freshBlood'] = None

      if 'hotStreak' in ranked_stats[i]:
        t = ranked_stats[i]['hotStreak']
        data['hotStreak'] = t
      else:
        data['hotStreak'] = None
else:
  data['tierRank'] = None
  data['wins'] = None
  data['losses'] = None
  data['veteran'] = None
  data['inactive'] = None
  data['freshBlood'] = None
  data['hotStreak'] = None

In [76]:
data


{'name': 'taehyungbottoms',
 'summonerLevel': 95,
 'tierRank': None,
 'wins': None,
 'losses': None,
 'veteran': None,
 'inactive': None,
 'freshBlood': None,
 'hotStreak': None}